## Quantitaive and qualitative data inspection

In [ ]:
# libraries 
import numpy as np
import time
import h5py
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
import matplotlib.animation as animation
from IPython.display import display

from utils.evaluate_utils import get_indices

### Qualitative analysis

In [ ]:
h5_files = ['../data/HR/HR_1.h5', '../data/HR/HR_2.h5', '../data/HR/HR_3.h5', '../data/HR/HR_4.h5', '../data/HR/HR_5.h5']
file_label = ['HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5']

# columns to load from data
colnames = ['u', 'v', 'w']


# load data
data = {}
for h5_file, label in zip(h5_files, file_label):
    with h5py.File(h5_file, 'r') as f:
        for vel in colnames:
            data[f'{label}_{vel}'] = np.array(f[vel]).squeeze()


In [ ]:
#TODO: add vmin and vmax to the plot
# make use to also choose the colname (data) such that magnitude and velocity can be plotted next to each other

# show interactive plot

T, X, Y, Z= data['HR_1_u'].shape


@widgets.interact(frame=(0, T), x = (0, X-1), vel=colnames, axis=[0, 1, 2])
def f(frame=5, x = 10, vel = "u", axis = 0):
    plt.figure(figsize=(10,10))
    N = len(h5_files)
    idxs = get_indices(frame,axis, x)

    for i, t_name in enumerate(file_label):

        plt.subplot(1, N, i+1)
        plt.imshow(data[t_name][vel][idxs])#,  vmin = min_v[vel[0]], vmax = max_v[vel[0]], cmap='viridis')
        plt.title(t_name)
        plt.axis('off')
    